In [1]:
# # ================================
# # پیش‌بینی 7 روز آینده با ورودی 1 ماه اخیر (RandomForestRegressor)
# # ================================
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings
# warnings.filterwarnings("ignore")

# import os
# from datetime import datetime
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.ensemble import RandomForestRegressor

# plt.rcParams['figure.figsize'] = (16, 10)
# sns.set_style("whitegrid")

# # ================================
# # مسیرها
# # ================================
# INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"
# OUTPUT_EXCEL = r"C:\BI\forecast_main_lube_oil_pump_press_RF.xlsx"
# OUTPUT_PLOTS = r"C:\BI\plots_timeseries_rf\\"
# LOG_FILE = r"C:\BI\regression_assetid_9286_rf_log.txt"
# os.makedirs(r"C:\BI", exist_ok=True)
# os.makedirs(OUTPUT_PLOTS, exist_ok=True)

# def log(msg):
#     print(msg)
#     with open(LOG_FILE, "a", encoding="utf-8") as f:
#         f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

# log("شروع اجرای مدل RandomForest برای پیش‌بینی 7 روز آینده با ورودی 1 ماه اخیر")

# # ================================
# # خواندن داده و ساخت datetime
# # ================================
# df = pd.read_excel(INPUT_FILE)
# log(f"داده خوانده شد → {len(df):,} ردیف")

# df["datetime"] = pd.to_datetime(df["RecordDate"] + " " + df["RecordTime"], errors="coerce")
# df = df.sort_values("datetime").reset_index(drop=True)

# # ================================
# # نام‌های خوانا
# # ================================
# friendly_names = {
#     "AssetID_8341": "Lube oil filter diffrential pressure",
#     "AssetID_8342": "Lube oil tank level",
#     "AssetID_8343": "Lube oil tank temperature",
#     "AssetID_8344": "Lube oil tank vapor pressure",
#     "AssetID_8346": "Over speed pressure",
#     "AssetID_9286": "Main lube oil pump press",   # هدف
#     "AssetID_9287": "Lube oil line pressure",
# }
# df.rename(columns=friendly_names, inplace=True)

# TARGET_COL = "Main lube oil pump press"
# if TARGET_COL not in df.columns:
#     raise ValueError(f"ستون هدف '{TARGET_COL}' در فایل ورودی یافت نشد.")

# # ================================
# # آماده‌سازی سری زمانی روزانه
# # ================================
# ts = df.dropna(subset=[TARGET_COL]).set_index("datetime")[TARGET_COL]
# ts = ts[ts >= 0]  # حذف مقادیر منفی
# daily = ts.resample("D").mean().interpolate()

# if len(daily) < 60:
#     raise ValueError("طول سری روزانه کمتر از 60 است؛ برای آموزش پایدار داده کافی نیست.")

# log(f"سری روزانه: {len(daily)} روز از {daily.index.min().date()} تا {daily.index.max().date()}")

# # ================================
# # ساخت ویژگی‌های lag برای ورودی 30 روز
# # ================================
# SEQ_LEN = 30     # ورودی: یک ماه اخیر
# HORIZON = 7      # خروجی: 7 روز آینده

# values = daily.values.reshape(-1, 1)

# # نرمال‌سازی
# scaler = MinMaxScaler()
# scaled = scaler.fit_transform(values)

# X, Y = [], []
# for i in range(len(scaled) - SEQ_LEN - HORIZON + 1):
#     X.append(scaled[i:i+SEQ_LEN].flatten())
#     Y.append(scaled[i+SEQ_LEN:i+SEQ_LEN+HORIZON].flatten())
# X, Y = np.array(X), np.array(Y)

# log(f"تعداد نمونه‌های آموزشی: {X.shape[0]:,} | شکل X: {X.shape} | شکل Y: {Y.shape}")

# # ================================
# # مدل RandomForestRegressor
# # ================================
# model = RandomForestRegressor(
#     n_estimators=300,
#     max_depth=12,
#     random_state=42,
#     n_jobs=-1
# )
# model.fit(X, Y)
# log("مدل RandomForest آموزش دید")

# # ================================
# # پیش‌بینی 7 روز آینده با آخرین 30 روز
# # ================================
# last_seq = scaled[-SEQ_LEN:].flatten().reshape(1,-1)
# pred_scaled = model.predict(last_seq)[0]
# pred_values = scaler.inverse_transform(pred_scaled.reshape(-1,1)).flatten()

# future_dates = pd.date_range(start=daily.index[-1] + pd.Timedelta(days=1), periods=HORIZON, freq="D")
# forecast_df = pd.DataFrame({"datetime": future_dates, "forecast_rf": pred_values})

# log("پیش‌بینی 7 روز آینده ساخته شد")

# # ================================
# # رسم نمودار
# # ================================
# plt.figure(figsize=(16, 8))
# plt.plot(daily.index, daily.values, label="واقعی (روزانه)", color="steelblue")
# plt.plot(forecast_df["datetime"], forecast_df["forecast_rf"], label="پیش‌بینی 7 روز آینده (RandomForest)", color="green", linewidth=2)
# plt.title("Main lube oil pump press - پیش‌بینی 7 روز آینده با ورودی 1 ماه اخیر (RandomForest)")
# plt.xlabel("تاریخ")
# plt.ylabel("فشار")
# plt.legend()
# plt.tight_layout()
# plt.savefig(os.path.join(OUTPUT_PLOTS, "rf_7day_forecast.png"), dpi=200, bbox_inches="tight")
# plt.close()

# # ================================
# # ذخیره خروجی در اکسل
# # ================================
# with pd.ExcelWriter(OUTPUT_EXCEL) as writer:
#     pd.DataFrame({"datetime": daily.index, "pressure": daily.values}).to_excel(writer, sheet_name="Daily_History", index=False)
#     forecast_df.to_excel(writer, sheet_name="7_Days_Forecast_RF", index=False)

# log("تمام شد! خروجی RandomForest ذخیره شد")
# print("\n" + "="*80)
# print("RandomForest forecast ساخته شد و ذخیره شد")
# print(f"فایل اکسل: {OUTPUT_EXCEL}")
# print(f"نمودارها: {OUTPUT_PLOTS}")
# print("="*80)


شروع اجرای مدل RandomForest برای پیش‌بینی 7 روز آینده با ورودی 1 ماه اخیر
داده خوانده شد → 10,927 ردیف
سری روزانه: 1690 روز از 2021-04-03 تا 2025-11-17
تعداد نمونه‌های آموزشی: 1,654 | شکل X: (1654, 30) | شکل Y: (1654, 7)
مدل RandomForest آموزش دید
پیش‌بینی 7 روز آینده ساخته شد
تمام شد! خروجی RandomForest ذخیره شد

RandomForest forecast ساخته شد و ذخیره شد
فایل اکسل: C:\BI\forecast_main_lube_oil_pump_press_RF.xlsx
نمودارها: C:\BI\plots_timeseries_rf\\


In [ ]:
# ================================
# پیش‌بینی 7 روز آینده با RandomForest برای 4 سیگنال روغن‌کاری
# ================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

plt.rcParams['figure.figsize'] = (16, 10)
sns.set_style("whitegrid")

# ================================
# مسیرها
# ================================
INPUT_FILE = r"C:\BI\KZCCPP\G11\Lube Oil System\lube_oil_system_data_g11.xlsx"
OUTPUT_EXCEL = r"C:\BI\KZCCPP\G11\Lube Oil System\forecast_lube_oil_params_RF.xlsx"
OUTPUT_PLOTS = r"C:\BI\KZCCPP\G11\Lube Oil System\plots_timeseries_rf\\"
LOG_FILE = r"C:\BI\KZCCPP\G11\Lube Oil System\regression_assetid_rf_log.txt"
os.makedirs(r"C:\BI\KZCCPP\G11\Lube Oil System", exist_ok=True)
os.makedirs(OUTPUT_PLOTS, exist_ok=True)

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

log("شروع اجرای مدل RandomForest برای 4 سیگنال و پیش‌بینی 7 روز آینده")

# ================================
# خواندن داده و ساخت datetime
# ================================
df = pd.read_excel(INPUT_FILE)
log(f"داده خوانده شد → {len(df):,} ردیف")

df["datetime"] = pd.to_datetime(df["RecordDate"] + " " + df["RecordTime"], errors="coerce")
df = df.sort_values("datetime").reset_index(drop=True)

# ================================
# نام‌های خوانا
# ================================
friendly_names = {
    "AssetID_8341": "Lube oil filter diffrential pressure",
    "AssetID_8342": "Lube oil tank level",
    "AssetID_8343": "Lube oil tank temperature",
    "AssetID_8344": "Lube oil tank vapor pressure",
    "AssetID_8346": "Over speed pressure",
    "AssetID_9286": "Main lube oil pump press",
    "AssetID_9287": "Lube oil line pressure",
}
df.rename(columns=friendly_names, inplace=True)

# ================================
# سیگنال‌های هدف
# ================================
target_cols = [
    "Main lube oil pump press",
    "Lube oil line pressure",
    "Lube oil tank temperature",
    "Lube oil tank level"
]

SEQ_LEN = 30
HORIZON = 7

# ================================
# اجرای مدل برای هر سیگنال
# ================================
with pd.ExcelWriter(OUTPUT_EXCEL) as writer:
    for col in target_cols:
        log(f"شروع پردازش سیگنال: {col}")

        ts = df.dropna(subset=[col]).set_index("datetime")[col]
        ts = ts[ts >= 0]  # حذف مقادیر منفی
        daily = ts.resample("D").mean().interpolate()

        if len(daily) < 60:
            log(f"سیگنال '{col}' داده کافی ندارد → {len(daily)} روز")
            continue

        log(f"سری روزانه '{col}': {len(daily)} روز از {daily.index.min().date()} تا {daily.index.max().date()}")

        # نرمال‌سازی
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(daily.values.reshape(-1, 1))

        # ساخت داده‌های آموزشی
        X, Y = [], []
        for i in range(len(scaled) - SEQ_LEN - HORIZON + 1):
            X.append(scaled[i:i+SEQ_LEN].flatten())
            Y.append(scaled[i+SEQ_LEN:i+SEQ_LEN+HORIZON].flatten())
        X, Y = np.array(X), np.array(Y)

        log(f"آموزش مدل RandomForest برای '{col}' | نمونه‌ها: {X.shape[0]}")

        model = RandomForestRegressor(
            n_estimators=300,
            max_depth=12,
            random_state=42,
            n_jobs=-1
        )
        model.fit(X, Y)

        # پیش‌بینی 7 روز آینده
        last_seq = scaled[-SEQ_LEN:].flatten().reshape(1, -1)
        pred_scaled = model.predict(last_seq)[0]
        pred_values = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()

        future_dates = pd.date_range(start=daily.index[-1] + pd.Timedelta(days=1), periods=HORIZON, freq="D")
        forecast_df = pd.DataFrame({
            "datetime": future_dates,
            f"{col} forecast": pred_values
        })

        # ذخیره در اکسل
        pd.DataFrame({"datetime": daily.index, col: daily.values}).to_excel(writer, sheet_name=f"{col}_History", index=False)
        forecast_df.to_excel(writer, sheet_name=f"{col}_Forecast_RF", index=False)

        # رسم نمودار
        plt.figure(figsize=(16, 8))
        plt.plot(daily.index, daily.values, label="واقعی (روزانه)", color="steelblue")
        plt.plot(forecast_df["datetime"], forecast_df[f"{col} forecast"], label="پیش‌بینی 7 روز آینده (RandomForest)", color="green", linewidth=2)
        plt.title(f"{col} - پیش‌بینی 7 روز آینده با ورودی 1 ماه اخیر (RandomForest)")
        plt.xlabel("تاریخ")
        plt.ylabel("مقدار")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_PLOTS, f"rf_7day_{col.replace(' ', '_')}.png"), dpi=200, bbox_inches="tight")
        plt.close()

log("تمام شد! خروجی RandomForest برای 4 سیگنال ذخیره شد")
print("\n" + "="*80)
print("RandomForest forecasts ساخته شد و ذخیره شد")
print(f"فایل اکسل: {OUTPUT_EXCEL}")
print(f"نمودارها: {OUTPUT_PLOTS}")
print("="*80)


شروع اجرای مدل RandomForest برای 4 سیگنال و پیش‌بینی 7 روز آینده
داده خوانده شد → 10,927 ردیف
شروع پردازش سیگنال: Main lube oil pump press
سری روزانه 'Main lube oil pump press': 1690 روز از 2021-04-03 تا 2025-11-17
آموزش مدل RandomForest برای 'Main lube oil pump press' | نمونه‌ها: 1654
شروع پردازش سیگنال: Lube oil line pressure
سری روزانه 'Lube oil line pressure': 1690 روز از 2021-04-03 تا 2025-11-17
آموزش مدل RandomForest برای 'Lube oil line pressure' | نمونه‌ها: 1654
شروع پردازش سیگنال: Lube oil tank temperature
سری روزانه 'Lube oil tank temperature': 1690 روز از 2021-04-03 تا 2025-11-17
آموزش مدل RandomForest برای 'Lube oil tank temperature' | نمونه‌ها: 1654
شروع پردازش سیگنال: Lube oil tank level
سری روزانه 'Lube oil tank level': 1690 روز از 2021-04-03 تا 2025-11-17
آموزش مدل RandomForest برای 'Lube oil tank level' | نمونه‌ها: 1654
تمام شد! خروجی RandomForest برای 4 سیگنال ذخیره شد

RandomForest forecasts ساخته شد و ذخیره شد
فایل اکسل: C:\BI\forecast_lube_oil_params_RF.xlsx
نموداره